In [1]:
from mip import *
import datetime
import pandas as pd
import numpy as np
from EMS import *

In [54]:
# try test using Pchg diff instead of xchg diff
def EMS_1_Pchg_test(PARAM,energyfromgrid=0,energycost=0,profit=0,multibatt=1,chargebatt=0,smoothcharge=0):
    # this is for 2 battery only
    # input Resolution (min)
    # input Horizon (min)
    if energyfromgrid < 0 or energycost < 0 or profit < 0 or multibatt < 0 or chargebatt < 0 or smoothcharge < 0 :
        raise('Weight must > 0')
    elif PARAM['Horizon'] % PARAM['Resolution'] != 0 :
        raise('variables length must be integer')
    elif energyfromgrid*energycost + profit*energycost + energyfromgrid*profit != 0 :
        raise('You can only choose one of the three energy objectives')
    #------------ change unit
    fs = 1/PARAM['Resolution'] #sampling freq(1/min)
    h = PARAM['Horizon'] #optimization horizon(min)
    k = int(h*fs) #length of variable
    Resolution_HR = PARAM['Resolution'] /60 # resolution in Hr
    
    #------------------------------- variables -----------------------
    model = Model(solver_name=CBC)
    Pnet = model.add_var_tensor((k,),name = 'Pnet',lb = -float('inf'),ub = float('inf'),var_type = CONTINUOUS)    
    Pdchg =     model.add_var_tensor((k,PARAM['battery']['num_batt']),name = 'Pdchg',lb = 0,ub = float('inf'),var_type = CONTINUOUS)
    xdchg =     model.add_var_tensor((k,PARAM['battery']['num_batt']),name = 'xdchg',lb = 0,ub = 1,var_type = INTEGER)
    Pchg =     model.add_var_tensor((k,PARAM['battery']['num_batt']),name = 'Pchg',lb = 0,ub = float('inf'),var_type = CONTINUOUS)
    xchg =     model.add_var_tensor((k,PARAM['battery']['num_batt']),name = 'xchg',lb = 0,ub = 1,var_type = INTEGER)
    soc =       model.add_var_tensor((k+1,PARAM['battery']['num_batt']),name = 'soc',lb = PARAM['battery']['min'][0],ub = PARAM['battery']['max'][0],var_type = CONTINUOUS)
    
    #model.objective = minimize(sum(u1) + sum(s1))
    if energyfromgrid > 0 :
        u1 =     model.add_var_tensor((k,),name = 'u1',lb = 0,ub = float('inf'),var_type = CONTINUOUS)
        obj_fcn = energyfromgrid*xsum(u1)
        model += (-np.eye(k) @ Pnet) <= u1
    if energycost > 0 :
        u1 =     model.add_var_tensor((k,),name = 'u1',lb = 0,ub = float('inf'),var_type = CONTINUOUS)
        obj_fcn = energycost*xsum(u1) 
        model += (-Resolution_HR*PARAM['Buy_rate']*np.eye(k) @ Pnet) <= u1
    if profit > 0 :
        u1 =     model.add_var_tensor((k,),name = 'u1',lb = -float('inf'),ub = float('inf'),var_type = CONTINUOUS)
        obj_fcn = profit*xsum(u1)
        model += (-Resolution_HR*PARAM['Buy_rate']*np.eye(k) @ Pnet) <= u1
        model += (-Resolution_HR*PARAM['Sell_rate']*np.eye(k) @ Pnet) <= u1
    if multibatt > 0 :
        s1 =     model.add_var_tensor((k,),name = 's1',lb = 0,ub = float('inf'),var_type = CONTINUOUS)
        obj_fcn += multibatt*xsum(s1)
        # force soc
        model += soc[1:,0] - soc[1:,1] <= s1
        model += -s1 <= soc[1:,0] - soc[1:,1]
    if chargebatt > 0 :
        for i in range(PARAM['battery']['num_batt']) :    
            obj_fcn += chargebatt*xsum((PARAM['battery']['max'][i] - soc[:,i])/(PARAM['battery']['max'][i] - PARAM['battery']['min'][i]))
    if smoothcharge > 0 :
        upper_bound_Pchg = model.add_var_tensor((k-1,PARAM['battery']['num_batt']),name = 'upper_bound_Pchg',lb = 0,ub = float('inf'),var_type = CONTINUOUS)
        upper_bound_Pdchg = model.add_var_tensor((k-1,PARAM['battery']['num_batt']),name = 'upper_bound_Pdchg',lb = 0,ub = float('inf'),var_type = CONTINUOUS)
        for i in range(PARAM['battery']['num_batt']) :
            obj_fcn += smoothcharge*( xsum(upper_bound_Pchg[:,i]) + xsum(upper_bound_Pdchg[:,i]) )
            model += Pchg[1:,i] - Pchg[:-1,i] <= upper_bound_Pchg[:,i]
            model += -upper_bound_Pchg[:,i] <= Pchg[1:,i] - Pchg[:-1,i]
            model += Pdchg[1:,i] - Pdchg[:-1,i] <= upper_bound_Pdchg[:,i]
            model += -upper_bound_Pdchg[:,i] <= Pdchg[1:,i] - Pdchg[:-1,i] 
           
    # try use pchg instead
    model.objective = minimize(obj_fcn)
    #------------------------------ constraint ----------------------  
   
    # battery constraint
    model += Pchg <= xchg*PARAM['battery']['charge_rate']
    model += Pdchg <= xdchg*PARAM['battery']['discharge_rate']
    model += xchg + xdchg <= 1
    model += 0 <= xchg + xdchg
    # Pnet constraint
    model += Pnet == PARAM['PV'] + Pdchg[:,0] + Pdchg[:,1] - PARAM['PL'] - Pchg[:,0] - Pchg[:,1]

    # battery dynamic constraint
    model += soc[0,:] == PARAM['battery']['initial']
    for i in range(PARAM['battery']['num_batt']) :
        model += soc[1:k+1,i] == (soc[0:k,i] 
        + (PARAM['battery']['charge_effiency'][i]*100*Resolution_HR / PARAM['battery']['actual_capacity'][i])*Pchg[0:k,i]
        - (Resolution_HR*100/(PARAM['battery']['discharge_effiency'][i]*PARAM['battery']['actual_capacity'][i]))*Pdchg[0:k,i])
    #print(f'num_row:{model.num_rows},num_col:{model.num_cols}')    
    #model.optimize(max_seconds_same_incumbent=2*60) # if feasible solution is found and not improve for 2 mins, terminates with that solution
    model.threads = -1 # use all available CPU threads
    model.preprocess = 1 # enable preprocess
    model.optimize()
    #----------------------------- solution ------------------
       
    sol = {}
    sol['datetime'] = pd.date_range(PARAM['Start_date'],PARAM['Start_date'] + datetime.timedelta(minutes=PARAM['Horizon']),freq=str(PARAM['Resolution']) +'min' )[:-1]
    sol['PARAM_PV'] = PARAM['PV']
    sol['PARAM_PL'] = PARAM['PL']
    sol['Buy_rate'] = PARAM['Buy_rate']
    sol['Sell_rate'] = PARAM['Sell_rate']
    sol['Pnet'] = [e.x for e in model.vars if  e.name[:4] == 'Pnet']
    sol['u1'] = [e.x for e in model.vars if  e.name[:2] == 'u1']
    sol['s1'] = [e.x for e in model.vars if  e.name[:2] == 's1']
    sol['Pchg_1'] = [e.x for e in model.vars if  e.name[:4] == 'Pchg' and e.name[-1] == '0']
    sol['Pchg_2'] = [e.x for e in model.vars if  e.name[:4] == 'Pchg' and e.name[-1] == '1']
    sol['Pdchg_1'] = [e.x for e in model.vars if e.name[:5] == 'Pdchg' and e.name[-1] == '0']
    sol['Pdchg_2'] = [e.x for e in model.vars if e.name[:5] == 'Pdchg' and e.name[-1] == '1']
    sol['xchg_1'] = [e.x for e in model.vars if e.name[:4] == 'xchg' and e.name[-1] == '0']
    sol['xchg_2'] = [e.x for e in model.vars if e.name[:4] == 'xchg' and e.name[-1] == '1']
    sol['xdchg_1'] = [e.x for e in model.vars if e.name[:5] == 'xdchg' and e.name[-1] == '0']
    sol['xdchg_2'] = [e.x for e in model.vars if e.name[:5] == 'xdchg' and e.name[-1] == '1']
    sol['soc_1'] = [e.x for e in model.vars if e.name[:3] == 'soc' and e.name[-1] == '0'][:-1]
    sol['soc_2'] = [e.x for e in model.vars if e.name[:3] == 'soc' and e.name[-1] == '1'][:-1]
    if smoothcharge > 0 :
        sol['upper_bound_Pchg'] = [e.x for e in model.vars if  e.name[:16] == 'upper_bound_Pchg' and e.name[-1] == '0'] + [0]
        sol['upper_bound_Pdchg'] = [e.x for e in model.vars if  e.name[:17] == 'upper_bound_Pdchg' and e.name[-1] == '0'] + [0]

    
    return pd.DataFrame(sol)

In [5]:
PARAM = {}
# add length check with res & horizon
PARAM['Horizon'] = 3*24*60        # horizon to optimize (min)
PARAM['Resolution'] = 15    # sampling period(min)
PARAM['PV_capacity'] = 50   # (kw) PV sizing for this EMS
TOU = getBuySellrate(Resolution=PARAM['Resolution'],
                                    Horizon=PARAM['Horizon'],
                                    TOU_CHOICE='THcurrent',
                                    start_time=datetime.timedelta(minutes=0))
PARAM['Buy_rate'] = TOU['buy'].to_numpy()
PARAM['Sell_rate'] = TOU['sell'].to_numpy()
PARAM['battery'] = {}
PARAM['battery']['charge_effiency'] = [0.95,0.95];              #  bes charge eff
PARAM['battery']['discharge_effiency'] = [0.95*0.93,0.95*0.93]; #  bes discharge eff note inverter eff 0.93-0.96
PARAM['battery']['discharge_rate'] = [30,30]; # kW max discharge rate
PARAM['battery']['charge_rate'] = [30,30]; # kW max charge rate
PARAM['battery']['actual_capacity'] = [125,125]; # kWh soc_capacity 
PARAM['battery']['initial'] = [20,20]; # userdefined int 0-100 %
PARAM['battery']['min'] = [20,20]; #min soc userdefined int 0-100 %
PARAM['battery']['max'] = [80,80]; #max soc userdefined int 0-100 %
PARAM['battery']['num_batt'] = len(PARAM['battery']['actual_capacity'])

In [3]:
root_folder = 'C:/Users/User/Desktop/VSCpython/EMS_on_production/input_data/historical/'
load_15min = pd.read_csv(root_folder + 'load_data_15minresample_concat.csv',parse_dates=['datetime'],usecols=['datetime','Ptot (kW)'])
pv_15min = pd.read_csv(root_folder + 'pv_data_15minresample_concat.csv',parse_dates=['datetime'],usecols=['datetime','Ptot (kW)'])
pv_scaling_factor = 50/8
# scale PV
pv_15min.iloc[:,1:] = pv_15min.iloc[:,1:]*pv_scaling_factor

In [6]:
PARAM['Start_date'] = pd.to_datetime('2023-03-16')

PARAM['PV']  = pv_15min[(pv_15min['datetime'].dt.date >= PARAM['Start_date']) & (pv_15min['datetime'].dt.date < PARAM['Start_date'] + pd.Timedelta(days=3))].iloc[:,1].to_numpy().flatten()
PARAM['PL']  = load_15min[(load_15min['datetime'].dt.date >= PARAM['Start_date']) &   (load_15min['datetime'].dt.date < PARAM['Start_date'] + pd.Timedelta(days=3))].iloc[:,1].to_numpy().flatten()

C:\Users\User\AppData\Local\Temp\ipykernel_6588\1427274596.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  PARAM['PV']  = pv_15min[(pv_15min['datetime'].dt.date >= PARAM['Start_date']) & (pv_15min['datetime'].dt.date < PARAM['Start_date'] + pd.Timedelta(days=3))].iloc[:,1].to_numpy().flatten()
C:\Users\User\AppData\Local\Temp\ipykernel_6588\1427274596.py:4: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  PARAM['PL']  = load_15min[(load_15min['datetime'].dt.date >= PARAM['Start_date']) &   (load_15min['datetime'].dt.date < PARAM['Start_date'] + pd.Timedelta(days=3))].iloc[:,1].to_numpy().fl

In [102]:
smoothcharge_weight = [0,0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09,
                        0.1,0.2,0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29,
                        0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                        1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70]
for w in smoothcharge_weight :
    print(f'begin optimizing w = {w}')
    sol_power = EMS_1_Pchg_test(PARAM,energycost=1,multibatt=1,chargebatt=1,smoothcharge=w)
    print(f'finish optimizing w = {w}')
    sol_power.to_csv(f'pareto_solution/sol_Pchg_w_{w}.csv',index=False)

begin optimizing w = 0.21000000000000002
finish optimizing w = 0.21000000000000002
begin optimizing w = 0.22
finish optimizing w = 0.22
begin optimizing w = 0.23
finish optimizing w = 0.23
begin optimizing w = 0.24000000000000002
finish optimizing w = 0.24000000000000002
begin optimizing w = 0.25
finish optimizing w = 0.25
begin optimizing w = 0.26
finish optimizing w = 0.26
begin optimizing w = 0.27
finish optimizing w = 0.27
begin optimizing w = 0.28
finish optimizing w = 0.28
begin optimizing w = 0.29000000000000004
finish optimizing w = 0.29000000000000004


In [7]:
# read solution
smoothcharge_weight =  [0,0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09,
                        0.1,0.2,0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29,
                        0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                        1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70]
expense_state = []
expense_power = []
state_change = []
power_change = []
for w in smoothcharge_weight :    
    sol_power = pd.read_csv(f'pareto_solution/sol_Pchg_w_{w}.csv')    
    expense_power.append(sol_power['u1'].sum())    
    if w != 0:
        power_change.append( sol_power['upper_bound_Pchg'].sum() +  sol_power['upper_bound_Pdchg'].sum())
    else :
        power_change.append( (abs(sol_power['Pchg_1'][1:].to_numpy() - sol_power['Pchg_1'][:-1].to_numpy()) ).sum()
                            + (abs(sol_power['Pdchg_1'][1:].to_numpy() - sol_power['Pdchg_1'][:-1].to_numpy()) ).sum()
                            ) 
                                

## plot solution

In [8]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
selected_weight = [0.01,0.2,0.3]
fig = make_subplots(rows=len(selected_weight), cols=1,subplot_titles=[f'smoothcharge_weight = {w}' for w in selected_weight],specs=[[{"secondary_y": True}] for e in selected_weight])
i = 1
for w in selected_weight :
    
    sol = pd.read_csv(f'pareto_solution/sol_Pchg_w_{w}.csv',parse_dates=['datetime'])
    excess_gen = sol['PARAM_PV'] - sol['PARAM_PL']
    fig.add_trace(
        go.Scatter(x=sol['datetime'], y=excess_gen, name = 'Excess gen',line={'color':'#000000'}),
        row=i, col=1
    )
   
    fig.add_trace(
        go.Scatter(x=sol['datetime'], y=sol['Pchg_1'], name = 'Pchg',mode='lines',line={'color':'#0000FF'}),
        row=i, col=1, secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=sol['datetime'], y=-sol['Pdchg_1'],name = 'Pdchg',mode='lines',line={'color':'#FF0000'}),
        row=i, col=1, secondary_y=True
    )
    fig.update_yaxes(title_text='Excess gen (kW)',row=i,col=1,range=[-20,20])

    fig.update_yaxes(title_text='Power (kW)',row=i,col=1,secondary_y=True,tickvals= [-10,0,10,20,30],range=[-10,30])
    
    i += 1
#fig.update_layout(height=1000, width=1500, title_text="",)    

fig.show()

## plot pareto curve

In [9]:
pd.DataFrame({'smooth_weight':smoothcharge_weight,'expense':expense_power,'power_change':power_change})

,smooth_weight,expense,power_change
0,0.00,358.876077,272.249193
1,0.01,358.876077,272.068699
2,0.02,358.876077,268.049546
3,0.03,359.300778,263.504381
4,0.04,359.684970,258.497128
5,0.05,359.795436,253.811291
6,0.06,359.795436,253.714139
7,0.07,359.886158,250.980818
8,0.08,359.959746,250.668571
9,0.09,360.204330,244.861983


In [13]:
import plotly.express as px
fig = px.line(y=expense_power, x=power_change,markers=True,
                 title='Pareto curve using Pchg, Pdchg as smoothcharge obj.',
                 #color=smoothcharge_weight
                 )
fig.update_yaxes(title_text='Expense (THB)')
fig.update_xaxes(title_text='Power difference (kW)')
fig.update_layout(height=1000, width=1500,plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        showgrid=True,
        gridcolor='black'
    ),
    yaxis=dict(
        linecolor='black',
        showgrid=True,
        gridcolor='black'
    )) 
fig.show()